# Download the module from Hugging Face 

The easiest way to download the model is by using Hugging Face. To configure the module, uncomment the line corresponding to the desired module. Ensure both HF_USER and HF_TOKEN are set as environment variables in the workbench.

You can also manually download the model files if your environment is not able to reach out to the internet. Navigate to the "Files and versions" tab on your Hugging Face module page (e.g., https://huggingface.co/ibm-granite/granite-7b-instruct/tree/main) and download the files. Use CLI tools or a browser to upload these files to your S3 bucket.

## Download Model from Huggingface



In [2]:
import os
# Choose the desired model
# my_repo = "huggingface.co/ibm-granite/granite-7b-instruct"
# my_repo = "huggingface.co/ibm/merlinite-7b"
# my_repo = "huggingface.co/instructlab/merlinite-7b-lab"
# my_repo = "https://huggingface.co/mistralai/Mistral-7B"
# my_repo = "https://huggingface.co/codellama/CodeLlama-7b-hf"
# my_repo = "https://huggingface.co/mosaicml/mpt-7b-chat"
# my_repo = "https://huggingface.co/mosaicml/mpt-7b-instruct"

# Ensure my_repo is set
my_repo = "huggingface.co/ibm-granite/granite-7b-instruct"  # Example default, change as needed
if not my_repo:
    raise ValueError("Please uncomment or set the 'my_repo' variable.")

# Prepare the GitHub repository URL with authentication
git_repo = f"https://{os.getenv('HF_USER')}:{os.getenv('HF_TOKEN')}@{my_repo}"

# Extract model directory name
model_dir = os.path.basename(my_repo)

# Clone or update the repository
if not os.path.exists(model_dir):
    !git clone $git_repo
else:
    %cd $model_dir
    !git fetch --all
    !git reset --hard origin/main
    %cd ..

print(f"Model Directory cloned in: {model_dir}")


Cloning into 'granite-7b-instruct'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 55 (delta 18), reused 0 (delta 0), pack-reused 1 (from 1)
Unpacking objects: 100% (55/55), 1.36 MiB | 8.91 MiB/s, done.
Filtering content: 100% (4/4), 4.55 GiB | 14.73 MiB/s, done.
Model Directory: granite-7b-instruct


## Configure S3 Bucket Environment Variables

In [4]:
import boto3
import botocore

# Fetch required environment variables
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.getenv('AWS_S3_ENDPOINT')
region_name = os.getenv('AWS_DEFAULT_REGION')
bucket_name = os.getenv('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise EnvironmentError("Ensure all AWS-related environment variables are properly set.")

# Create a Boto3 session
session = boto3.session.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Set up S3 resource with session
s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name
)

# Connect to the specified bucket
bucket = s3_resource.Bucket(bucket_name)
print(f"S3 connectivity established successfully for the bucket: {bucket_name}")


S3 connectivity established successfully for the bucket: mys3-1efde082-92ab-47ae-8b56-15e3b6fed5aa


## Upload model to S3 bucket

### Next Steps

Now that the model is uploaded to S3 storage, you can deploy your model configure the ServingRuntime and InferenceEndpoint from the Workbanch models tab.